# Wordle Solver in Python

#### By Yicheng Shen & Linxuan Wang

Our function works in a way that it accepts a `wordle.puzzle()` object, automatically guesses up to 6 times, and breaks once it guesses correctly. 

We choose "plaid" as the initial guess based on our domain knowledge. There are many potentially well-performing first guesses, such as "react", "salet", "other", etc. The rule of thumb is to have words with common letters and vowels. 

For each round, we store the positions at which "🟩"/"🟨"/"⬜" appears in the guess result as well as the corresponding letters. We also record and update letters that were guesses correctly and incorrectly. Bascially, our funtion looks up the list of possible words (starting from the full word bank). It would select words that have the correct letters in the "🟩" positions, have letters indicated in by "🟨" but not in those positions, and don't have letters that are ruled out by "⬜". After these three rounds of filtering, the possbile guesses left should all fit the criterion from previous results. 

After each guess we narrow down the list of possible answers based on information we stored. We pick a word among the highest commonality to be the next guess. The commonality is determined based on the frequency of each letter at each position among the remaining possible guesses. The intuition is to choose a common word whose letters appear "frequently" and is likely to be the answer. 

We also notice that most common failures of solving the wordle puzzles usually happen when the solver has confirmed four or three "🟩" letters and positions, but is stuck by a long list of possible guesses to fill the remaining "⬜" positions. In response to this situation, we add a mechanism that would make a different guess to eliminate several letters at a time. The guess should be a word containing letters that did not appear in previous guesses. 

Overall, our solver is able to solve random wordle puzzles approximately 90% of the time with an average number of guess being about 4.57. 

In [1]:
import wordle
import collections
from collections import Counter
from itertools import chain
import operator
    
def puzzle_solver(p):
    """ This function is built as a wordle solver using python. \n It should be used in accompany with the `wordle.py` and `wordle.txt` files. \n The input should be a world puzzle object. \n The output will return the status of the puzzle.
    """
    
    our_guess = "plaid" # chosen based on domain knowledge
    all_words = wordle.words.copy()
    possible_guess = wordle.words.copy()
    green_letter_all = set() # store all correctly guessed letters 
#    yellow_letter_all = set()
    wrong_letter_all = set() # store all incorrectly guessed letters 
    
    def calculate_word_commonality(word): # calcualte the commonality of a given word based on frequency of each letter
        score = 0.0
        for i in range(5):
            score += Letter_Frequency[i][word[i]]
        return score / (5 - len(set(word)) + 1)

    def sort_by_word_commonality(words): # calculate and sort the commonality of a given list of words
        sort_by = operator.itemgetter(1)
        return sorted(
            [(word, calculate_word_commonality(word)) for word in words],
            key=sort_by,
            reverse=True,
        )
    
    def sort_by_num_difference(words): # count and sort the number of unguessed of a given list of words
        sort_by = operator.itemgetter(1)
        return sorted(
            list(zip(all_words,num_differ)),
            key=sort_by,
            reverse=True,
        )
    
    
    for trial in range(1,7):
        
        guest_result = p.guess(our_guess)      
        
        # Below counts the number of appearance of letter at each position of the 5-letter word
        Letter_Counter = [Counter(chain.from_iterable([word[x] for word in possible_guess])) for x in range(5)]  
        
        # Below calculates the frequency of letter at each position of the 5-letter word
        Letter_Frequency = [{character: value / Letter_Counter[x].total() for character, value in Letter_Counter[x].items()} for x in range(5)] 

            
        if p.is_solved(): 
            break
        else: 
            green_pois = [x for x in range(5) if guest_result[x] == "🟩"]
            yellow_pois = [x for x in range(5) if guest_result[x] == "🟨"]
            gray_pois = [x for x in range(5) if guest_result[x] == "⬜"]
            
            green_letter = set([our_guess[i] for i in range(5) if i in set(green_pois)])
            green_letter_all.update((green_letter))
            
            yellow_letter = set([our_guess[i] for i in range(5) if i in set(yellow_pois)])
            gray_letter = set([our_guess[i] for i in range(5) if i in set(gray_pois)])
            
            wrong_letter = gray_letter - yellow_letter
            wrong_letter_all.update((wrong_letter))

            for i in green_pois: # filter possible_guess on green letters
                possible_guess = [x for x in possible_guess if 
                                  our_guess[i] == x[i]]  
                
            for j in gray_pois: # filter possible_guess on wrong letters
                possible_guess = [x for x in possible_guess if 
                                   x[j] not in wrong_letter]
                                
            for k in yellow_pois: # filter possible_guess on yellow letters
                possible_guess = [x for x in possible_guess if 
                                  our_guess[k] != x[k] and 
                                  our_guess[k] in set([x[j] for j in range(len(list(x))) if j not in set(green_pois)])]

#            if trial == 1:
#                our_guess = "other" alternative
            if "🟨" not in guest_result and trial in [1,2,3] and len(possible_guess) >= 3:
        
                # Chunk below sorts out the words that have the maximum number of unguessed letters, and use it as the next guess
                num_differ = [len(set(x).difference(wrong_letter_all,green_letter_all)) for x in all_words]
                max_diff = sort_by_num_difference(all_words)[0][1]
                eliminate_guess = [x for (x,y) in sort_by_num_difference(all_words) if y == max_diff][:1]
                our_guess = random.choice(eliminate_guess)              
            else: 
                narrow_guess = sort_by_word_commonality(possible_guess)[:4]
                our_guess = random.choice([x for (x,y) in narrow_guess])
    
    return p

In [2]:
print(puzzle_solver.__doc__)

 This function is built as a wordle solver using python. 
 It should be used in accompany with the `wordle.py` and `wordle.txt` files. 
 The input should be a world puzzle object. 
 The output will return the status of the puzzle.
    


## Assessment

In [3]:
import random
#random.seed(1234)
n = 100  # can change to a large number if needed

# Change both the seed value and `n` of puzzles to get 
# a more accurate view of your solver's performance.

puzzles  = [wordle.puzzle() for i in range(n)]
attempts = [puzzle_solver(p) for p in puzzles]
solved   = [p for p in puzzles if p.is_solved()]
n_guess  = [p.n_guesses() for p in solved]

print(f"Solved {len(solved)} of {len(puzzles)} puzzles attempted.")
if (len(solved) != 0):
    print(f"Average # of guesses required: {sum(n_guess)/len(n_guess)}")                         

Solved 96 of 100 puzzles attempted.
Average # of guesses required: 4.5
